# Data Prep Lab

In [11]:
import pandas as pd

In [12]:
fpc = "data/staged/calc_cases.csv"
fpm = "data/staged/mass_cases.csv"
fpd = "data/staged/dicom.csv"

In [14]:
dfc = pd.read_csv(fpc)
dfc['case_id'] = dfc['patient_id'] + "_" + dfc['abnormality_type']
dfc.head()

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,dataset,image_series_uid,roi_mask_series_uid,cropped_image_series_uid,case_id
0,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,train,1.3.6.1.4.1.9590.100.1.2.474143160103683865197...,1.3.6.1.4.1.9590.100.1.2.393344010211719049419...,1.3.6.1.4.1.9590.100.1.2.393344010211719049419...,P_00005_calcification
1,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,train,1.3.6.1.4.1.9590.100.1.2.250596608311207922527...,1.3.6.1.4.1.9590.100.1.2.296281207812130400303...,1.3.6.1.4.1.9590.100.1.2.296281207812130400303...,P_00005_calcification
2,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,train,1.3.6.1.4.1.9590.100.1.2.228699627313487111012...,1.3.6.1.4.1.9590.100.1.2.314135871111943890422...,1.3.6.1.4.1.9590.100.1.2.314135871111943890422...,P_00007_calcification
3,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,train,1.3.6.1.4.1.9590.100.1.2.104743410411133110629...,1.3.6.1.4.1.9590.100.1.2.914582796124855152034...,1.3.6.1.4.1.9590.100.1.2.914582796124855152034...,P_00007_calcification
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.406725628213826290127...,1.3.6.1.4.1.9590.100.1.2.281397494612871934937...,1.3.6.1.4.1.9590.100.1.2.281397494612871934937...,P_00008_calcification


In [3]:
dfc = pd.read_csv(fpc)
dfm = pd.read_csv(fpm)
cols = [col for col in dfc.columns if col in dfm.columns]
df = pd.concat([dfc[cols], dfm[cols]], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3568 entries, 0 to 1695
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   patient_id                3568 non-null   object
 1   breast_density            3568 non-null   int64 
 2   left_or_right_breast      3568 non-null   object
 3   image_view                3568 non-null   object
 4   abnormality_id            3568 non-null   int64 
 5   abnormality_type          3568 non-null   object
 6   assessment                3568 non-null   int64 
 7   pathology                 3568 non-null   object
 8   subtlety                  3568 non-null   int64 
 9   dataset                   3568 non-null   object
 10  image_series_uid          3568 non-null   object
 11  roi_mask_series_uid       3568 non-null   object
 12  cropped_image_series_uid  3568 non-null   object
dtypes: int64(4), object(9)
memory usage: 390.2+ KB


In [8]:
dfi = df.drop(columns=['roi_mask_series_uid', 'cropped_image_series_uid']).rename(columns={"image_series_uid": "series_uid"})
dfr = df.drop(columns=['image_series_uid', 'cropped_image_series_uid']).rename(columns={"roi_mask_series_uid": "series_uid"})
dfc = df.drop(columns=['roi_mask_series_uid', 'image_series_uid']).rename(columns={"cropped_image_series_uid": "series_uid"})
dfi['series_description'] = 'full mammogram images'
dfr['series_description'] = 'ROI mask images'
dfc['series_description'] = 'cropped images'
dfm = pd.concat([dfi,dfr,dfc], axis=0)
dfm = dfm.drop_duplicates(subset=['series_uid'])
dfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6775 entries, 0 to 1639
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   patient_id            6775 non-null   object
 1   breast_density        6775 non-null   int64 
 2   left_or_right_breast  6775 non-null   object
 3   image_view            6775 non-null   object
 4   abnormality_id        6775 non-null   int64 
 5   abnormality_type      6775 non-null   object
 6   assessment            6775 non-null   int64 
 7   pathology             6775 non-null   object
 8   subtlety              6775 non-null   int64 
 9   dataset               6775 non-null   object
 10  series_uid            6775 non-null   object
 11  series_description    6775 non-null   object
dtypes: int64(4), object(8)
memory usage: 688.1+ KB


In [10]:
dfd = pd.read_csv(fpd)
dfd.info()
dfm2 = dfm.drop(columns=['patient_id', 'left_or_right_breast', 'image_view'])

dfd2 = dfd.merge(dfm2, on='series_uid', how='left')
dfd2.info()
dtypes = {"height": 'int64',
"width": 'int64',
"bits": 'int64',
"smallest_image_pixel": 'int64',
"largest_image_pixel": 'int64',
"image_pixel_range": 'int64'}

dfd2.astype(dtypes)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10238 entries, 0 to 10237
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   series_uid            10238 non-null  object
 1   filepath              10238 non-null  object
 2   patient_id            10238 non-null  object
 3   side                  10238 non-null  object
 4   image_view            10238 non-null  object
 5   height                10238 non-null  int64 
 6   width                 10238 non-null  int64 
 7   bits                  10238 non-null  int64 
 8   smallest_image_pixel  10238 non-null  int64 
 9   largest_image_pixel   10238 non-null  int64 
 10  image_pixel_range     10238 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 880.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10238 entries, 0 to 10237
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                ---

,series_uid,filepath,patient_id,side,image_view,height,width,bits,smallest_image_pixel,largest_image_pixel,image_pixel_range,breast_density,abnormality_id,abnormality_type,assessment,pathology,subtlety,dataset,series_description
0,1.3.6.1.4.1.9590.100.1.2.373687394012570965941...,data/raw/CBIS-DDSM/Mass-Test_P_01378_RIGHT_CC/...,P_01378,RIGHT,CC,5206,3001,16,0,65535,65535,3,1,mass,0,BENIGN,5,test,full mammogram images
1,1.3.6.1.4.1.9590.100.1.2.144135582511225260304...,data/raw/CBIS-DDSM/Mass-Training_P_00335_LEFT_...,P_00335,LEFT,MLO,5986,2461,16,0,65535,65535,2,1,mass,5,MALIGNANT,5,train,full mammogram images
2,1.3.6.1.4.1.9590.100.1.2.362620324812321450233...,data/raw/CBIS-DDSM/Mass-Test_P_00820_LEFT_MLO/...,P_00820,LEFT,MLO,4680,2576,16,0,65535,65535,3,1,mass,5,MALIGNANT,5,test,full mammogram images
3,1.3.6.1.4.1.9590.100.1.2.270964980412538159307...,data/raw/CBIS-DDSM/Mass-Training_P_01632_RIGHT...,P_01632,RIGHT,CC,379,425,16,42260,65535,23275,1,1,mass,4,MALIGNANT,4,train,ROI mask images
4,1.3.6.1.4.1.9590.100.1.2.270964980412538159307...,data/raw/CBIS-DDSM/Mass-Training_P_01632_RIGHT...,P_01632,RIGHT,CC,4704,3312,8,0,255,255,1,1,mass,4,MALIGNANT,4,train,ROI mask images
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10233,1.3.6.1.4.1.9590.100.1.2.420516911812665689337...,data/raw/CBIS-DDSM/Mass-Training_P_00039_RIGHT...,P_00039,RIGHT,CC,547,464,16,0,65535,65535,2,1,mass,5,MALIGNANT,5,train,ROI mask images
10234,1.3.6.1.4.1.9590.100.1.2.420516911812665689337...,data/raw/CBIS-DDSM/Mass-Training_P_00039_RIGHT...,P_00039,RIGHT,CC,4688,2704,8,0,255,255,2,1,mass,5,MALIGNANT,5,train,ROI mask images
10235,1.3.6.1.4.1.9590.100.1.2.164919549013310446318...,data/raw/CBIS-DDSM/Calc-Training_P_02190_RIGHT...,P_02190,RIGHT,CC,5251,2341,16,0,65535,65535,3,1,calcification,3,MALIGNANT,1,train,full mammogram images
10236,1.3.6.1.4.1.9590.100.1.2.176964796612485740037...,data/raw/CBIS-DDSM/Mass-Training_P_00475_RIGHT...,P_00475,RIGHT,MLO,4744,2632,16,0,65535,65535,3,1,mass,4,BENIGN,4,train,full mammogram images


In [1]:
a = [1,2,4,5]
[b for b in range(6) not in a]

TypeError: 'bool' object is not iterable